In [3]:
#@title ## Base imports
import os
import sys
import numpy as np
import scipy
import pandas as pd
import geopandas
import plotly.express as px

import sklearn
import sklearn.linear_model
import statsmodels.api as sm

## Load ENT county df (specifically wide-type df) from csv file
To understand what is meant by long type and wife type dataframes, see https://towardsdatascience.com/visualization-with-plotly-express-comprehensive-guide-eb5ee4b50b57

In [4]:
df_bill_orig = pd.read_csv("data/2022_05_05 sums and slopes ent with HCPCS descriptions.csv", 
                           dtype={
                               "HCPCS Code": str,
                               "Total Number of Services": np.int64,
                               **{f"Total Number of Services: {year}": np.int64 for year in range(2015,2019+1)}
                               })  # gets per healthcare code info

In [5]:
df_bill_orig.head(2)

,HCPCS Code,HCPCS Description,Total Number of Services,Total Medicare Payment Amount,Total Number of Services: 2019,Total Medicare Payment Amount: 2019,Total Number of Services: 2018,Total Medicare Payment Amount: 2018,Total Number of Services: 2017,Total Medicare Payment Amount: 2017,Total Number of Services: 2016,Total Medicare Payment Amount: 2016,Total Number of Services: 2015,Total Medicare Payment Amount: 2015,Total Medicare Payment Amount: Slope,Total Medicare Payment Amount: Pearson Coef,Total Number of Services: Slope,Total Number of Services: Pearson Coef
0,11042,Removal of skin and tissue first 20 sq cm or less,46073,4237372.20,6537,644733.10,11483,1317431.90,14608,1165644.85,8266,600263.11,5179,509299.24,0.000002,0.425707,0.000101,0.245111
1,11642,Removal of malignant growth (1.1 to 2.0 centim...,14749,1201135.31,2634,214496.56,2585,213254.24,2923,238474.17,3055,245483.24,3552,289427.10,-0.000047,-0.929111,-0.003798,-0.935792


In [12]:
df_bill_wide = df_bill_orig.set_index(["HCPCS Code", "HCPCS Description"])
# Rename the columns so they can be split  easier. The 20 is the first two digits of the year columns
df_bill_wide.columns = [col.replace(": ",": : ").replace(": 20","Annual: 20") for col in df_bill_wide.columns]
# Multiindex
df_bill_wide.columns = pd.MultiIndex.from_tuples([tuple(col.split(": ")) if ":" in col else (col,"","Sum") for col in df_bill_wide.columns], names=["Category","Stat","Year"])
df_bill_wide = df_bill_wide[sorted(df_bill_wide)]  # rearrange cols alphabetically
df_bill_wide = df_bill_wide.sort_values(by=("Total Number of Services","","Sum"), ascending=False)  # sort rows by volume 
categories = df_bill_wide.columns.levels[0]  #["Total Number of Services", "Total Medicare Payment Amount"]

The slope given in the csv file is actually the inverse slope. We need to either recalculate it or

In [63]:
def calc_slope(y, x, invert=False ):
    if invert: 
        # the inverse linear regression does not necessarily have 1/slope of the regular linear regression
        # the "2022_05_05 sums and slopes ent with HCPCS descriptions.csv" contains the inverse linear regression
        temp = x
        x = y 
        y = temp
    regress = scipy.stats.linregress(x, y=y)  # x=np.arange(2015,2019+1)
    return {"Slope": regress.slope, "Pearson Coef": regress.rvalue, "P": regress.pvalue}
    #return {"Slope2": regress.slope, "Pearson Coef2": regress.rvalue, "P": regress.pvalue}
    #return {"Slope": regress.slope, "Pearson Coef": regress.rvalue, "Intercept": regress.intercept, "P": regress.pvalue}

In [64]:
df_bill_wide_stats = df_bill_wide.copy()
for category in categories:
    new_df = df_bill_wide[(category,"Annual")].apply(calc_slope,axis=1, result_type="expand", args=(np.arange(2015,2019+1),) )
    df_bill_wide_stats[[(category,"",new_col) for new_col in new_df.columns ]]=new_df
    #df_bill_wide[(category,"","Slope")]=df_bill_wide[(category,"Annual")].apply(calc_slope,axis=1)
df_bill_wide_stats = df_bill_wide_stats[sorted(df_bill_wide_stats.columns)]  # rearrange cols alphabetically

In [65]:
df_bill_wide_stats.head(2)

Category                                                      Total Medicare Payment Amount  ... Total Number of Services
Stat                                                                                         ...                   Annual
Year                                                                                      P  ...                     2019
HCPCS Code HCPCS Description                                                                 ...                         
31575      Diagnostic examination of voice box using flexi...                      0.135205  ...                    62112
31231      Diagnostic examination of nasal passages using ...                      0.093238  ...                    42188

[2 rows x 18 columns]

In [54]:
1/-222163.937	

-4.50118058539807e-06

In [45]:
df_bill_wide

Category                                                      Total Medicare Payment Amount  ... Total Number of Services
Stat                                                                                         ...                   Annual
Year                                                                           Pearson Coef  ...                     2019
HCPCS Code HCPCS Description                                                                 ...                         
31575      Diagnostic examination of voice box using flexi...                 -7.608948e-01  ...                    62112
31231      Diagnostic examination of nasal passages using ...                  8.144384e-01  ...                    42188
14060      Tissue transfer repair of wound (10 sq centimet...                 -6.440928e-01  ...                    12781
30140      Removal of nasal air passage                                       -4.595438e-01  ...                    14256
31579      Examination to assess movement of vocal cord fl...                  2.649743e-01  ...                    15023
30520      Reshaping of nasal cartilage                                        8.823509e-01  ...                    10934
11042      Removal of skin and tissue first 20 sq cm or less                   4.257069e-01  ...                     6537
31267      Removal of nasal sinus tissue using an endoscope                    9.239828e-01  ...                    10126
14040      Tissue transfer repair of wound (10 sq centimet...                 -9.218206e-01  ...                     6854
15260      Relocation of patient skin to nose, ears, eyeli...                 -5.065926e-01  ...                     6439
31525      Diagnostic examination of voice box using an en...                  9.365166e-01  ...                     9470
31276      Exploration of nasal sinus using an endoscope                      -6.900806e-01  ...                     3838
14301      Tissue transfer repair of wound (30.1 to 60.0 s...                  9.514447e-01  ...                     5027
31237      Biopsy or removal of nasal polyp or tissue usin...                  9.433822e-01  ...                     5431
14041      Tissue transfer repair of wound (10.1 to 30.0 s...                  2.400251e-01  ...                     4311
69436      Incision of eardrum with insertion of eardrum t...                 -7.003179e-01  ...                     4032
15004      Preparation of graft site of face, scalp, eyeli...                  9.651560e-01  ...                     4141
31256      Incision of nasal sinus using an endoscope                         -5.065444e-01  ...                     3465
11642      Removal of malignant growth (1.1 to 2.0 centime...                 -9.291114e-01  ...                     2634
69930      Implantation of cochlear device                                     9.836189e-01  ...                     2677
60500      Removal or exploration of parathyroid glands                        8.636788e-01  ...                     2644
31571      Injection of vocal cords using an endoscope wit...                 -2.647500e-02  ...                     1859
31622      Diagnostic examination of lung airways using an...                 -9.045496e-01  ...                     1750
31541      Removal of growth of tongue and/or vocal cord s...                 -3.971829e-01  ...                     1349
11643      Removal of malignant growth (2.1 to 3.0 centime...                 -8.779105e-01  ...                     1182
69801      Incision of fluid canals of inner ear with infu...                 -8.392964e-01  ...                     1106
31536      Biopsy of voice box using an endoscope with ope...                 -5.238394e-03  ...                     1190
69433      Incision of eardrum with insertion of eardrum t...                 -9.818496e-02  ...                     1113
30930      Therapeutic fracture of nasal passages                              3.368906e-01  ...